In [26]:
import numpy as np
from scipy.special import factorial
from scipy import linalg as LA
import scipy.sparse as sps
from scipy.linalg import eigh
from scipy.special import eval_hermite
from scipy.signal import argrelextrema

# Solucionando de la ecuacion Schrödinger con Deep Learning


En este cuaderno, buscaremos las soluciones de la ecuación de Schrödinger para múltiples sistemas. Para ello, utilizaremos modelos de Deep Learning para cada sistema.

Comenzaremos por encontrar los estados estacionarios de la ecuación de Schrödinger independiente del tiempo:

$$
\Big( - \frac{\hbar^2}{2m} \frac{\partial^2}{\partial x^2} + V(x) \Big) \phi_n(x) = E_n \phi_n(x) 
$$

Donde $\hbar$ es la constante de Plank reducida, $m$ es la masa de la partícula, $V(x)$ es el potencial bajo el cual evoluciona la partícula. $\phi_n(x)$ es el $n$-th estado estacionario del sistema cuántico, con energía $E_n$. El valor $n$ puede ser discreto ($n \in \mathbb{Z}$) o continuo ($n \in \mathbb{R}$).

Las funciones de onda estacionaria $\phi_n(x)$ tener una evolución trivial en el tiempo:

$$
\phi_n(x,t) = \phi_n(x) e^{-i \frac{E_n}{\hbar}t}
$$

Forman una base del espacio de Hilbert del hamiltoniano del sistema.

$$
\hat{H} = \frac{\hat{p}^2}{2m} + V(\hat{x})
$$

Por tanto, la evolución de un estado arbitrario del sistema $\psi(x,t)$ será dado por:

$$
\psi(x,t) = \sum_k c_k \phi_k(x) e^{-i \frac{E_k}{\hbar}t} 
$$

Donde $c_k$ son coeficientes lineales y la suma de $k$ puede ser discreta o continua (una integral). El módulo cuadrado de los coeficientes $|c_k|^2$ representa la probabilidad de estar en el $k$-th estado exitado. 

### La base del Oscilador Harmonico

Elegimos como base de $\mathcal{H}$ las funciones propias del oscilador armónico con $m=1$, $\hbar=1$ y $\omega=1$. 

$$
\phi_n(x) = \frac{1}{\sqrt{n!2^n \sqrt{\pi}}} e^{-x^2/2} H_n(x)
$$

Donde $ H_n(x) $ es el $n$ th Polinomio de Hermite, que se define por la recurrencia:

$$
H_n(x) = 2xH_{n-1}(x) - 2n H_{n-2}(x)\\
H_0(x) = 1
$$

Ya que $\{\phi_n(x)\}_n$ forma una base de $\mathcal{H}$, podemos escribir cualquier función de onda
 $\psi(x)$ como una combinación lineal de las funciones propias:

$$
\psi(x) = \sum_{n=0}^\infty a_n \phi_n(x)
$$

Por tanto, la energía media de $\psi$ es

$$
<H> = <\psi|\hat{H}|\psi> = \int_{-\infty}^\infty \Big(\sum_{n=0}^\infty a_n \phi_n(x)\Big) \hat{H} \Big(\sum_{m=0}^\infty a_m \phi_m(x)\Big) dx =\\
\sum_{n=0}^\infty \sum_{m=0}^\infty a_n a_m \int_{-\infty}^\infty \phi_n(x) H(x) \phi_m(x)dx = \sum_{n=0}^\infty \sum_{m=0}^\infty a_n a_m C_{nm}
$$

donde

$$
C_{nm} = \int_{-\infty}^\infty A_n e^{-x^2/2} H_n(x) \Big(-\frac{1}{2} \frac{\partial^2}{\partial x^2} + V(x) \Big) A_m e^{-x^2/2} H_m(x) dx, \quad A_n = \frac{1}{\sqrt{n!2^n \sqrt{\pi}}}
$$

Para encontrar una buena estimación de la energía del estado fundamental, generaremos una base finita de autoestados $\{\phi_n\}_{n=0}^N$ de H.O y luego encuentra los coeficientes $\{a_n\}$ que minimizan la energía media $<H>$. 

## Encontrando los coeficientes $\{a_n\}$

Para encontrar los coeficientes $\{a_n\}$ que minimizan la energía $<H>$ usamos el Teorema de los multiplicadores de Lagrange, que establece que el mínimo local de una función $F$, bajo una restricción $G$ es la solución de



$$
\nabla F = \lambda \nabla G, \quad \lambda \in \mathbb{R}
$$

En este caso $F$ es la energía media

$$
F(\{a_n\}) = <H>
$$

y $G$ es la restricción de normalización. Ya que $\{\phi_n\}$ es una base de  $\mathcal{H}$, entonces

$$
G(\{a_n\}) = \sum_{n=0}^N a_n^2 = 1
$$

Ahora tenemos que calcular las derivadas parciales de $F$ y $G$.

$$
\frac{\partial F}{\partial a_i} = \frac{\partial}{\partial a_i} \Big( \sum_{n=0}^N \sum_{m=0}^N a_n a_m C_{nm} \Big)=  \frac{\partial}{\partial a_i} \Big( \sum_{n=0} a_n \Big) \Big(\sum_{m=0}^N a_m C_{nm}\Big) + \frac{\partial}{\partial a_i} \Big( \sum_{m=0} a_m \Big) \Big(\sum_{n=0}^N a_n C_{nm}\Big) =\\
\sum_{m=0}^N a_m C_{im} + \sum_{n=0}^N a_n C_{ni} = \sum_{n=0}^N a_n (C_{in} + C_{ni})
$$

Por tanto, el gradiente de $F$ es un sistema lineal:

$$
\nabla F(\vec{a}) = D \vec{a}, \quad \vec{a} = 
\begin{pmatrix} 
a_1\\
\vdots\\
a_N
\end{pmatrix}, \quad D \in \mathcal{M}_{N}(\mathbb{R}), \ [D]_{ij} = C_{ij} + C_{ji}
$$

Entonces,

$$
\frac{\partial G}{\partial a_i} = \frac{\partial }{\partial a_i} \Big(\sum_{n=0}^N a_n^2\Big) = 2a_i
$$

Por lo tanto, la ecuación del multiplicador de Lagrange se convierte en

$$
\nabla F(\vec{a}) = \lambda \nabla G(\vec{a}) \Longleftrightarrow D \vec{a} = 2\lambda \vec{a}
$$

Lo cual es un problema de valores propios. La solución se encontrará resolviendo el problema de valores propios y luego seleccionando el vector $\vec{a_0}$ el cual minimiza $<H>$. Ya que la base de $\phi_n(x)$ es finito (tomamos hasta $N$ funciones), la solución será una aproximación del verdadero vector propio. Cuando $N \rightarrow \infty$ la solucion $\psi(x)$ convergerá al estado fundamental de $H$ . Además, dado que los vectores propios de
 $D$ son ortonormales, el vector con el $n$ th la energía más baja será una aproximación a la $n$ th estado emocionado del hamiltoniano. 

In [ ]:
def Calcular_energia_alpha_minimo(alphas, estado_n=0):
    N = len(alphas)
    # Generando la matriz C_nm
    C = np.zeros((N,N))
    for n in range(N):
        for m in range(N):
            C[n,m] = C_nm(n,m,alphas)

    # Generando la matriz D
    D = np.zeros((N,N))
    for n in range(N):
        for m in range(n+1):
            D[n,m] = C[n,m] + C[m,n]
            D[m,n] = D[n,m]

    # Diagonalizando la matriz D
    vaps, veps = eigh(D)

    # Calcular <H> 
    Hs = np.zeros(N)
    for i in range(N):
        a = veps[:, i]
        for n in range(N):
            for m in range(N):
                Hs[i]+=a[n]*a[m]*C[n,m]

    # 4. We choose the vector which minimizes <H>
    # If n_state!=0, we choose the vector with n_state-th lowest energy
    # as an approximation of the n_state excited state 
    sel = np.argsort(Hs)[estado_n] #np.argmin(Hs)
    a = veps[:, sel] # Final value of eigenvalues for state n_state
    E_a = Hs[sel] # Value of the energy
    return E_a, a

## Calculating $C_{nm}$

Para diagonalizar la matriz $D$, necesitamos calcular los coeficientes $C_{nm}$

$$
C_{nm} = A_nA_m \int_{-\infty}^\infty e^{-x^2/2} H_n(x) (-\frac{1}{2} \frac{\partial^2}{\partial x^2} + V(x)) H_m(x) e^{-x^2/2} dx
$$
$$
A_n = \frac{1}{\sqrt{n! 2^n \sqrt{\pi}}}
$$

Para hacerlo, necesitamos calcular:

$$
\frac{\partial^2}{\partial x^2}(H_m(x) e^{-x^2/2} ) = e^{-x^2/2}\Big((x^2-1) H_m(x) - 4mx H_{m-1}(x) + 4m(m-1)H_{m-2}(x)\Big) := e^{-x^2/2} P(x)
$$

$$
C_{nm} = A_n A_m \Big( - \frac{1}{2} \int_{-\infty}^\infty  H_n(x) P(x) e^{-x^2} dx + \int_{-\infty}^\infty e^{-x^2} H_n(x)H_m(x)V(x) dx\Big) = \\
A_nA_m\Big(- \frac{1}{2} I(n,m,2) + 1/2 I(n,m,0) + 2mI(n,m-1,1) - 2m(m-1)I(n, m-2, 0) + I_V\Big)
$$

Donde $I_V$ es la integral correspondiente al potencial $V(x)$. Si

$$
V(x) = \sum_{i=1}^N \alpha_i x^i
$$

Entonces

$$
I_V = \sum_{i=1}^N \alpha_i I(n,m,i)
$$


In [ ]:
def C_nm(n, m, alphas):
    An = 1./np.sqrt(np.sqrt(np.pi)*factorial(n)*2**n)
    Am = 1./np.sqrt(np.sqrt(np.pi)*factorial(m)*2**m)
    I1 = -1/2*I_nmr(n,m,2)
    I2 = 1/2*I_nmr(n,m,0)
    I3 = 2*m*I_nmr(n,m-1,1)
    I4 = -2*m*(m-1)*I_nmr(n, m-2,0)
    Iv = 0
    for i in range(len(alphas)):
        Iv+=alphas[i]*I_nmr(n,m,i)
    return An*Am*(I1+I2+I3+I4+Iv)

## Integrando los polinomios de Hermite

Con el fin de generar una base del espacio de Hilbert que usaremos para aproximar las funciones de onda del estado general, necesitamos realizar algunas integrales que involucren polinomios de Hermite:

$$
I(n,m,r) = \int_{-\infty}^\infty x^r e^{-x^2} H_n(x) H_m(x) dx
$$

Donde $H_n(x)$ es el $n$-th Polinomio de Hermite, definido por la recurrencia:

$$
H_n(x) = \frac{1}{2x}\Big(H_{n+1}(x) + 2nH_{n-1}(x)\Big)\\
H_0(x) = 1
$$

Debido a las limitaciones de normalización del oscilador armónico, sabemos que:

$$
I(n,m,0) = \sqrt{\pi} 2^n n! \delta_{n,m} 
$$

Por lo tanto,

$$
I(n,m,r) = \int_{-\infty}^\infty x^r e^{-x^2} H_n(x) H_m(x) dx = \int_{-\infty}^\infty x^r e^{-x^2}\frac{1}{2x}\Big(H_{n+1}(x) + 2nH_{n-1}(x)\Big)H_m(x)dx = \\
\frac{1}{2}I(n+1,m,r-1) + nI(n-1,m,r-1)
$$

Usando esta recurrencia, calcularemos los valores de $I(n,m,r)$.

In [ ]:
def I_nmr(n, m, r):
    if r<0 or n<0 or m<0:
        return 0
    if r==0:
        if n==m:
            return np.sqrt(np.pi)*2**n*factorial(n)
        else:
            return 0
    return 1./2*I_nmr(n+1,m,r-1) + n*I_nmr(n-1,m,r-1)

$$
\psi(x) = \sum_{n=0}^\infty a_n \phi_n(x)
$$

In [ ]:
def final_wavefunction( xmin, xmax, n_points, a):
    x = np.arange(xmin, xmax, (xmax - xmin)/n_points)
    n_samples, _ = a.shape
    # Construct matrix of phi_n
    phis = np.zeros((N, n_points))

    for i in range(N):
        phis[i,:] = HO_wavefunction(i, xmin, xmax, n_points)

    waves = np.zeros((n_samples, n_points))
    for i in range(n_samples):
        for j in range(n_points):
            waves[i,j] = np.dot(a[i,:],phis[:,j])
        # convention: To choose the phase we make the maximums be first
        w = waves[i,:]
        maxi = argrelextrema(w, np.greater)[0] #array con indices de maximos localles de w
        mini = argrelextrema(w, np.less)[0]    #array con indices de minimos localles de w
        idx2= np.abs(w[maxi])>5e-2 
        maxi = maxi[idx2] #idices con valores absolutos maximos de la funcion wave > 0.05
        idx2= np.abs(w[mini])>5e-2 
        mini = mini[idx2] #idices con valores absolutos minimos de la funcion wave > 0.05
        if len(maxi)==0 and len(mini)>0:
            waves[i,:] = -waves[i,:]
        elif len(mini)>0 and len(maxi)>0 and mini[0]<maxi[0]:
            waves[i,:] = -waves[i,:]
    return waves, x, phis

$$
\phi_n(x) = \frac{1}{\sqrt{n!2^n \sqrt{\pi}}} e^{-x^2/2} H_n(x)
$$

In [ ]:
def HO_wavefunction(n, xmin, xmax, n_points):
    x = np.arange(xmin, xmax, (xmax - xmin)/n_points)
    herm = eval_hermite(n, x) # H_n(x)
    exp = np.exp(- x**2/2) # Exponential term
    phi_n = exp*herm

    # Normalization
    h = (xmax - xmin)/n_points
    C = 1./np.sqrt(np.sum(phi_n*phi_n*h))
    phi_n = C*phi_n
    
    return phi_n

$$
V(x) = \sum_{i=1}^k \alpha_i x^i
$$

In [ ]:
def evaluar_potencial(self, xmin, xmax, n_points, alpha):
    x = np.arange(xmin, xmax, (xmax - xmin)/n_points)
    n_samples, k = alpha.shape
    V = np.zeros((n_samples, n_points))
    x_mat = (x**np.arange(k)[:,None])# Matrix of powers of x: x^0, x^1, x^2, ..., x^N (in every row)
    V = np.zeros((n_samples, n_points))# V(x) in each row different alpha
    for i in range(n_samples):
        for j in range(n_points):
            V[i,j] = np.dot(alpha[i,:],x_mat[:,j])
    return V, x